In [1]:
#!pip install wikipedia langchain BeautifulSoup

In [2]:
from langchain import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.agents import Tool, tool
from langchain import agents
from bs4 import BeautifulSoup
import langchain
langchain.debug = False
import os
import requests

os.environ['OPENAI_API_KEY'] = 'YOUR_VALID_OPENAI_KEY'

In [3]:
# defining a tool that can interact with PubChem and obtain materials properties
def find_property(compound, property):
    '''This function queries the compound from pubchecm and obtains property of the compund.
    Use this tool if you do not know a property of a compound.
    example use: find_property('KOH', 'Melting Point) '''
    try:
        # query the PubChem database
        r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/' + compound +'/cids/JSON')
        response = r.json()
        cid = response["IdentifierList"]["CID"][0]
        data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/xml")
        html = BeautifulSoup(data.content, "xml")
        tag = html.find(name='TOCHeading', string=property.title()).find_next_sibling('Information').find(name='String').string
        return tag
    except:
        print(f"Could not find the {property} for {compound} on PubChem.")

from langchain.tools import StructuredTool
tools = [StructuredTool.from_function(find_property)]

In [4]:
from langchain.memory import ConversationBufferMemory
llm = OpenAI(
        model_name='gpt-4',
        temperature=0,
    )

memory = ConversationBufferMemory()
agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True, memory=memory,  
    )

C:\Users\mehra\anaconda3\envs\agent\lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\mehra\anaconda3\envs\agent\lib\site-packages\langchain\llms\openai.py:789: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
agent.run(input="Experiment: Heat up methanol up to its boiling point.\
            Check for Health Hazards of the compound and stop the experiment if unsafe.\
            Experiment conditions: No fume hood available.")



> Entering new AgentExecutor chain...
The user wants to heat up methanol to its boiling point and wants to know the health hazards associated with methanol. Since I don't know the boiling point and health hazards of methanol, I will use the find_property tool to find this information.
Action:
```
{
  "action": "find_property",
  "action_input": {
    "compound": "methanol",
    "property": "Boiling Point"
  }
}
```
Observation: 148.3 °F at 760 mmHg (NTP, 1992)
Thought:Now that I have the boiling point of methanol, I need to find out the health hazards associated with it. I will use the find_property tool again to find this information. 
Action:
```
{
  "action": "find_property",
  "action_input": {
    "compound": "methanol",
    "property": "Health Hazards"
  }
}
```
Observation: Exposure to excessive vapor causes eye irritation, head- ache, fatigue and drowsiness. High concentrations can produce central nervous system depression and optic nerve damage. 50,000 ppm will probably caus

'The boiling point of methanol is 148.3 °F at 760 mmHg. However, methanol can be hazardous to health. Exposure to excessive vapor can cause eye irritation, headache, fatigue, and drowsiness. High concentrations can produce central nervous system depression and optic nerve damage. It can be fatal in high concentrations and can be absorbed through the skin. Swallowing may cause death or eye damage. Given these health hazards, it is not safe to conduct this experiment without a fume hood. Please ensure proper safety measures are in place before proceeding with such experiments.'